# Preprocessing

This notebook loads the UCI Adult dataset, applies lightweight but robust preprocessing, and prepares two artifacts:

- `adult_clean.csv`: cleaned, consistency-checked dataset for EDA.
- `adult_model_ready.csv`: modelling-ready dataset with missing values handled and duplicates removed.


## Cell 1 - Libraries


In [22]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

## Cell 2 - Configuration, helpers and raw import


In [23]:
print(os.getcwd())

# Project paths (notebook is expected to live in `notebooks/`)
PROJECT_ROOT = Path("..").resolve()
DATA_RAW_DIR = PROJECT_ROOT / "data" / "raw" / "adult"

# Canonical column names for the UCI Adult dataset
COLUMN_NAMES = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "sex",
    "capital_gain", "capital_loss", "hours_per_week", "native_country",
    "income",
]

NUMERIC_COLUMNS = [
    "age",
    "fnlwgt",
    "education_num",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
]

TARGET_COLUMN = "income"
SPLIT_COLUMN = "split"


        
def load_adult_splits(raw_dir: Path) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Load train and test splits of the Adult dataset from the raw directory.

    - Treat "?" (with or without leading space) as missing.
    - Normalize labels in the test split so that `income` is exactly "<=50K" or ">50K".
    - Add a `split` column to keep track of origin (train/test).
    """
    train_path = raw_dir / "adult.data"
    test_path = raw_dir / "adult.test"

    print(f"Reading train from: {train_path}")
    print(f"Reading test from : {test_path}")

    common_read_csv_kwargs = {
        "header": None,
        "names": COLUMN_NAMES,
        "na_values": ["?", " ?"],  # catch "?" as missing
        "sep": ",",
        "skipinitialspace": True,
    }

    df_train = pd.read_csv(train_path, **common_read_csv_kwargs)
    df_test = pd.read_csv(test_path, skiprows=1, **common_read_csv_kwargs)  # skip header/comment line

    # Normalise label formatting in test split (e.g. "<=50K." -> "<=50K")
    df_test[TARGET_COLUMN] = (
        df_test[TARGET_COLUMN]
        .astype(str)
        .str.replace(".", "", regex=False)
        .str.strip()
    )

    df_train[SPLIT_COLUMN] = "train"
    df_test[SPLIT_COLUMN] = "test"

    return df_train, df_test


def summarise_dataframe(df: pd.DataFrame, name: str = "df") -> pd.DataFrame:
    """Print a compact summary (shape, target distribution, missingness, duplicates).

    Returns a DataFrame with missing-value statistics per column.
    """
    print(f"{name} shape: {df.shape}")

    print("\nTarget distribution (income):")
    dist = (
        df[TARGET_COLUMN]
        .value_counts(dropna=False)
        .to_frame("count")
        .assign(proportion=lambda s: (s["count"] / len(df)).round(4))
    )
    print(dist)

    missing_counts = df.isna().sum()
    missing_pct = (missing_counts / len(df) * 100).round(2)
    summary = (
        pd.DataFrame({"n_missing": missing_counts, "pct_missing": missing_pct})
        .sort_values("pct_missing", ascending=False)
    )

    print(f"\nNumber of duplicated rows: {df.duplicated().sum()}")
    return summary


# Load raw train and test splits
df_train_raw, df_test_raw = load_adult_splits(DATA_RAW_DIR)

# Combine into a single dataframe
df_raw = pd.concat([df_train_raw, df_test_raw], ignore_index=True)

print(f"\nTrain shape: {df_train_raw.shape}")
print(f"Test shape : {df_test_raw.shape}")
print(f"Combined   : {df_raw.shape}")

df_raw.head()

/Users/villafuertech/Documents/Academic/University/Septimo_Semestre/Trusthworthy_ML/Projects/3_Project/fairness-project/notebooks
Reading train from: /Users/villafuertech/Documents/Academic/University/Septimo_Semestre/Trusthworthy_ML/Projects/3_Project/fairness-project/data/raw/adult/adult.data
Reading test from : /Users/villafuertech/Documents/Academic/University/Septimo_Semestre/Trusthworthy_ML/Projects/3_Project/fairness-project/data/raw/adult/adult.test

Train shape: (32561, 16)
Test shape : (16281, 16)
Combined   : (48842, 16)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,split
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,train
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,train
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,train
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,train
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,train


## Cell 3 - Basic cleaning (EDA-ready)

We create a cleaned version of the combined dataset:

- Strip whitespace from string columns.
- Enforce numeric dtypes for numeric columns.
- Keep the `split` column for traceability.


In [24]:
# Create a cleaned version of the combined dataframe suitable for EDA
df_clean = df_raw.copy()

# Strip leading/trailing whitespace in all string columns
str_cols = df_clean.select_dtypes(include="object").columns
df_clean[str_cols] = df_clean[str_cols].apply(lambda col: col.str.strip())

# Ensure numeric columns have numeric dtype
df_clean[NUMERIC_COLUMNS] = df_clean[NUMERIC_COLUMNS].apply(
    pd.to_numeric,
    errors="coerce",  # if something unexpected sneaks in, convert to NaN
)

df_clean.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income            object
split             object
dtype: object

## Cell 4 - Data quality checks

High-level overview of the cleaned dataframe:

- Shape.
- Target distribution.
- Missing values per column.
- Number of duplicates.


In [25]:
missing_summary = summarise_dataframe(df_clean, name="df_clean")
missing_summary

df_clean shape: (48842, 16)

Target distribution (income):
        count  proportion
income                   
<=50K   37155      0.7607
>50K    11687      0.2393

Number of duplicated rows: 29


,n_missing,pct_missing
occupation,2809,5.75
workclass,2799,5.73
native_country,857,1.75
age,0,0.00
fnlwgt,0,0.00
education,0,0.00
education_num,0,0.00
marital_status,0,0.00
relationship,0,0.00
race,0,0.00


## Cell 5 - Create modelling-ready dataset and persist artifacts

We create a second, modelling-focused version of the dataset where:

- Rows with missing values in key socio-economic fields (`workclass`, `occupation`, `native_country`) are dropped.
- Exact duplicate rows are dropped.
- Both EDA-ready and modelling-ready datasets are saved under `data/processed/adult/`.


In [26]:
# Columns that must not be missing for standard modelling
MODEL_NA_SUBSET = ["workclass", "occupation", "native_country"]

# Drop rows with missing values in the selected subset and remove duplicates
df_model = (
    df_clean
    .dropna(subset=MODEL_NA_SUBSET)
    .drop_duplicates()
    .reset_index(drop=True)
)

print(f"df_clean shape       : {df_clean.shape}")
print(f"df_model_ready shape : {df_model.shape}")
print(f"Rows dropped for modelling: {len(df_clean) - len(df_model)}")

# Persist artifacts
DATA_PROCESSED_DIR = PROJECT_ROOT / "data" / "processed" / "adult"
DATA_PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

eda_path = DATA_PROCESSED_DIR / "adult_clean.csv"          # EDA-ready
model_path = DATA_PROCESSED_DIR / "adult_model_ready.csv"   # modelling-ready

df_clean.to_csv(eda_path, index=False)
df_model.to_csv(model_path, index=False)

print(f"\nSaved EDA-ready dataset to   : {eda_path}")
print(f"Saved model-ready dataset to : {model_path}")

df_clean shape       : (48842, 16)
df_model_ready shape : (45194, 16)
Rows dropped for modelling: 3648

Saved EDA-ready dataset to   : /Users/villafuertech/Documents/Academic/University/Septimo_Semestre/Trusthworthy_ML/Projects/3_Project/fairness-project/data/processed/adult/adult_clean.csv
Saved model-ready dataset to : /Users/villafuertech/Documents/Academic/University/Septimo_Semestre/Trusthworthy_ML/Projects/3_Project/fairness-project/data/processed/adult/adult_model_ready.csv
